In [1]:
#Importing Packages
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn import tree
import re
from pprint import pprint

In [2]:
#Loading datasets
data1=pd.read_csv("C:/Users/Taron/Desktop/Vahe/Car Copart/General/cars_price.csv")
data2=pd.read_csv("C:/Users/Taron/Desktop/Vahe/Car Copart/General/salesdata1.csv")
result = pd.merge(data1, data2, on='VIN')
result.columns

Index(['Unnamed: 0', 'VIN', 'Price', 'Year', 'Make', 'Model Group',
       'Model Detail', 'Body Style', 'Color', 'Damage Description', 'Odometer',
       'Est. Retail Value', 'Repair cost', 'Engine', 'Drive', 'Cylinders',
       'Sale Status'],
      dtype='object')

In [3]:
del result['Unnamed: 0']

In [4]:
result = result.dropna()
result = result[result.Price != 0]
result = result[result.Price != "UNKNOWN"]

In [5]:
result.shape

(4027, 16)

In [6]:
result.loc[:,"Price"]=[re.sub("USD", "", i) for i in result.loc[:,"Price"]]
result.loc[:,"Price"]=[re.sub("CAD", "", i) for i in result.loc[:,"Price"]]
result.loc[:,"Price"]=[re.sub(r"\$", "", i) for i in result.loc[:,"Price"]]
result.loc[:,"Price"]=[re.sub(r" ", "", i) for i in result.loc[:,"Price"]]

In [7]:
data=result.loc[:, ["Price", 'Make', 'Model Group','Color', 
                  'Damage Description', 'Sale Status',
                  'Year','Odometer','Est. Retail Value']]

In [8]:
data.head()

,Price,Make,Model Group,Color,Damage Description,Sale Status,Year,Odometer,Est. Retail Value
0,1300,VOLKSWAGEN,ROUTAN SE,WHITE,Front,Pure,2010,80311,9671
1,2550,TOYOTA,COROLLA BA,RED,Side,Pure,2010,112976,7570
2,4350,ACURA,TSX,WHITE,Side,Pure,2010,96183,9245
3,4100,ACURA,MDX,SILVER,Rear,Pure,2010,111533,15747
4,900,TOYOTA,COROLLA BA,GRAY,Front,Pure,2010,232125,5840


In [9]:
data.loc[:, "Price"] = data["Price"].astype('int')

In [10]:
data.dtypes

Price                  int32
Make                  object
Model Group           object
Color                 object
Damage Description    object
Sale Status           object
Year                   int64
Odometer               int64
Est. Retail Value      int64
dtype: object

In [11]:
Toyota=data[data.Make=="TOYOTA"]
Toyota.shape

(1036, 9)

In [12]:
X1=Toyota.iloc[:, 2:]
X2=pd.read_csv("C:/Users/Taron/Desktop/corolla small.csv")
Y=Toyota.iloc[:, 0]

In [13]:
X=pd.concat([X1, X2], ignore_index=True)

In [14]:
categorical_feature_mask=X.dtypes==object
categorical_cols=X.columns[categorical_feature_mask].tolist()

In [15]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
le=LabelEncoder()
X[categorical_cols]=X[categorical_cols].astype(str)

In [16]:
X[categorical_cols]=X[categorical_cols].apply(lambda col:
                                                   le.fit_transform(col))
X[categorical_cols].head()

,Model Group,Color,Damage Description,Sale Status
0,6,12,7,2
1,6,7,2,2
2,6,2,7,2
3,13,13,2,2
4,6,6,2,2


In [17]:
n=[len(np.unique(X[i])) for i in [
    'Model Group','Color','Damage Description','Sale Status']]

In [18]:
n

[31, 19, 11, 3]

In [19]:
sum(n)

64

In [20]:
X.head()

,Model Group,Color,Damage Description,Sale Status,Year,Odometer,Est. Retail Value
0,6,12,7,2,2010,112976,7570.0
1,6,7,2,2,2010,232125,5840.0
2,6,2,7,2,2010,141307,5978.0
3,13,13,2,2,2010,146093,10372.0
4,6,6,2,2,2010,123135,7353.0


In [24]:
#Encoding categorical data
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(categorical_features = categorical_feature_mask)
X=ohe.fit_transform(X).toarray()

# DECISION TREE

In [25]:
X_train, X_test, y_train,y_test=train_test_split(X[0:1036,k], Y, test_size=0.2, random_state=0)

In [26]:
regressor=DecisionTreeRegressor(criterion="mse", random_state=0,
                                  max_depth=3, min_samples_leaf=3)
regressor.fit(X_train, y_train)

DecisionTreeRegressor(criterion='mse', max_depth=3, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=3,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=0, splitter='best')

In [27]:
y_pred=regressor.predict(X_test)
y_pred[0:10]

array([3359.69551282, 5316.0130719 , 3359.69551282, 2092.33128834,
       3359.69551282, 5316.0130719 , 2092.33128834, 3359.69551282,
       3359.69551282, 5316.0130719 ])

In [28]:
score = regressor.score(X_test, y_test)
score

0.6089418771636481

# LINEAR REGRESSION

In [21]:
k=list(range(0,67))
for i in [0,31,50,61]:
    k.remove(i)

In [21]:
to_be_pred=X[1036:,k]

In [22]:
to_be_pred

array([[    0.,     0.,     0., ...,  2012., 26467.,  9993.],
       [    0.,     0.,     0., ...,  2013., 92019.,  8435.],
       [    0.,     0.,     0., ...,  2013., 60890.,  9996.],
       ...,
       [    0.,     0.,     0., ...,  2013., 81946.,  8536.],
       [    0.,     0.,     0., ...,  2013., 55563., 11247.],
       [    0.,     0.,     0., ...,  2012., 85300.,  8496.]])

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X[0:1036,k],Y,test_size=0.2, random_state=0)

In [24]:
from sklearn.linear_model import LinearRegression
regressor=LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [25]:
y_pred=regressor.predict(X_test)

In [26]:
from sklearn.metrics import r2_score
r2_score(y_test,y_pred)

0.7211465328653388

In [27]:
X_test.shape

(208, 63)

In [28]:
to_be_pred.shape

(28, 63)

Corolla prediciton

In [29]:
pred=regressor.predict(to_be_pred)

In [30]:
pred

array([2652.57274969, 2664.71811799, 2685.59657333, 2339.82721583,
       2993.25583131, 2353.92740896, 2399.53326732, 2820.7282251 ,
       3163.37828231, 2359.83038434, 2796.84007799, 3424.76201603,
       2804.56253554, 2873.44383715, 2517.51728334, 2016.5787199 ,
       3882.71706284, 3147.49348306, 2891.33507364, 3126.05497512,
       4139.48523694, 2894.63160499, 3834.95671267, 2587.77277812,
       3054.42804677, 3089.9654682 , 3451.1339373 , 3311.79485118])

In [32]:
X2["Price"]=pred
corolla_full=X2

In [33]:
corolla_full

,Model Group,Color,Damage Description,Sale Status,Year,Odometer,Est. Retail Value,Price
0,COROLLA BA,SILVER,Front,Minimum,2012,26467,9993.00,2652.572750
1,COROLLA BA,BLACK,Front,Minimum,2013,92019,8435.00,2664.718118
2,COROLLA BA,BLUE,Front,Minimum,2013,60890,9996.00,2685.596573
3,COROLLA BA,BLUE,Front,Minimum,2012,83581,9597.00,2339.827216
4,COROLLA BA,GRAY,Front,Minimum,2013,63958,10518.00,2993.255831
5,COROLLA BA,BLACK,Front,Minimum,2012,89688,8346.00,2353.927409
6,COROLLA BA,BLUE,Front,Pure,2012,67531,9084.00,2399.533267
7,COROLLA BA,SILVER,Front,Minimum,2013,94010,9036.00,2820.728225
8,COROLLA BA,RED,Front,Pure,2013,42246,11142.00,3163.378282
9,COROLLA BA,BLACK,Front,Minimum,2012,78428,8440.00,2359.830384
